# Tb Mean

In [ ]:
import pandas as pd
features_and_target = pd.read_csv('data_with_everything.csv')


In [ ]:

features_and_target.columns

In [ ]:
final_score_list=[]
for val in features_and_target['Tw_TB_mean']:
    if val > 0:
        final_score_list.append(1)
    elif val < 0:
        final_score_list.append(-1)
    else:
        final_score_list.append(0)

print(len(final_score_list))

features_and_target = features_and_target.assign(class_label_Tb_mean=final_score_list)
 

In [ ]:
features_and_target.columns

In [ ]:

features_and_target_to_use = features_and_target[['Scopus_Subjects','title_TB','abstract_TB',\
                                                  'Count_HashTags','Abstract_Length','followers_count','author_count',\
                                                  'class_label_Tb_mean']]

In [ ]:
# observing the counts of positive and negative
from collections import Counter
tw_Tb_mean_class_list =features_and_target_to_use['class_label_Tb_mean'].tolist()
a = dict(Counter(tw_Tb_mean_class_list))
print(a)

In [ ]:
features_and_target_to_use.shape
features_and_target_to_use_2_labels = features_and_target_to_use

In [ ]:
'''
# considering only +1 and -1
features_and_target_to_use_2_labels = features_and_target_to_use[features_and_target_to_use.class_label_Tb_mean != 0]
'''

In [ ]:
features_and_target_to_use_2_labels.shape

In [ ]:
# observing the counts of positive and negative
from collections import Counter
tw_Tb_mean_class_list =features_and_target_to_use_2_labels['class_label_Tb_mean'].tolist()
a = dict(Counter(tw_Tb_mean_class_list))
print(a)

In [ ]:
features_and_target_to_use_2_labels.columns  = ['Scopus Subjects', 'Title Sentiment','Abstract Sentiment', 'Hashtag Count','Abstract Length','Tweet Reach','Author Count','Tweet Sentiment']

In [ ]:
features_and_target_to_use_2_labels.columns

In [ ]:
features_and_target_to_use_2_labels

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn
corrMatrix = features_and_target_to_use_2_labels.corr()
sn.heatmap(corrMatrix, annot=True, cmap='coolwarm')
plt.xticks(rotation=45)
plt.show()

In [ ]:
'''
# Seaborn visualization library
import seaborn as sns
# Create the default pairplot
sns.pairplot(features_and_target_to_use_2_labels, hue='class_label_Tb_mean')
'''

Standard scaling

In [ ]:
print(features_and_target_to_use_2_labels['Scopus Subjects'].unique())

print(len(features_and_target_to_use_2_labels['Scopus Subjects'].unique()))

In [ ]:
# taking features
X = features_and_target_to_use_2_labels.iloc[:, :-1].values
y = features_and_target_to_use_2_labels.iloc[:, 7].values

# Encoding categorical data
# Encoding the Independent Variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()
X[:, 0] = labelencoder_X.fit_transform(X[:, 0])
onehotencoder = OneHotEncoder(categorical_features = [0])
X = onehotencoder.fit_transform(X).toarray()

from imblearn.over_sampling import SMOTE
import numpy as np
# observing the counts of positive and negative before SMOTE
unique_elements, counts_elements = np.unique(y, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))
# applying SMOTE
smt = SMOTE()
X, y = smt.fit_sample(X, y)
# checking the rows
# observing the counts of positive and negative after SMOTE
unique_elements, counts_elements = np.unique(y, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))


# splitiing the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


##Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)




# Fitting Decision Tree Classification to the Training set
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier(criterion='entropy', max_depth=14, min_samples_split=600)
DT.fit(X_train, y_train)

# Predicting the Test set results
y_predDT = DT.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predDT)

import sklearn.metrics
print('Accuracy',sklearn.metrics.accuracy_score(y_test, y_predDT))
#print('Precision',sklearn.metrics.precision_score(y_test, y_predDT))
#print('Recall',sklearn.metrics.recall_score(y_test, y_predDT))
#print('F-1 score',sklearn.metrics.f1_score(y_test, y_predDT))

print()
print('confusion matrix')
print(cm)
print()

# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = DT, X = X_train, y = y_train, cv = 10)
print('accuracies mean',accuracies.mean())
print('accuracies with 2 SD', accuracies.std() * 2)
print()

# Applying Grid Search to find the best model and the best parameters
from sklearn.model_selection import GridSearchCV

parameters = [{'criterion': ["gini"], 'max_depth': [1,2,3,14,15], 'min_samples_split': [4,6,8,300,450,600]},
               {'criterion': ["entropy"], 'max_depth': [1,2,13,14,15], 'min_samples_split': [4,6,250,425,450,475,575,600]}]
grid_search = GridSearchCV(estimator = DT,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1, verbose=1)
grid_search = grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_


from sklearn.metrics import classification_report
report =classification_report(y_test, y_predDT)

print()
print('classification report')
print(report)
print()

print('best accuracy', best_accuracy)
print('best parameters')
print(best_parameters)
print()

print('feature importances')
print(features_and_target_to_use_2_labels.columns)
print(DT.feature_importances_)
print()

In [ ]:
# taking features
X = features_and_target_to_use_2_labels.iloc[:, :-1].values
y = features_and_target_to_use_2_labels.iloc[:, 7].values

# Encoding categorical data
# Encoding the Independent Variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()
X[:, 0] = labelencoder_X.fit_transform(X[:, 0])
onehotencoder = OneHotEncoder(categorical_features = [0])
X = onehotencoder.fit_transform(X).toarray()

from imblearn.over_sampling import SMOTE
import numpy as np
# observing the counts of positive and negative before SMOTE
unique_elements, counts_elements = np.unique(y, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))
# applying SMOTE
smt = SMOTE()
X, y = smt.fit_sample(X, y)
# checking the rows
# observing the counts of positive and negative after SMOTE
unique_elements, counts_elements = np.unique(y, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))


# splitiing the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


##Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)


# Fitting Random Forest Classification to the Training set
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth= 120, min_samples_split=80, n_estimators=40)
RF.fit(X_train, y_train)

# Predicting the Test set results
y_predRF = RF.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predRF)


import sklearn.metrics
print('Accuracy',sklearn.metrics.accuracy_score(y_test, y_predRF))
#print('Precision',sklearn.metrics.precision_score(y_test, y_predRF))
#print('Recall',sklearn.metrics.recall_score(y_test, y_predRF))
#print('F-1 score',sklearn.metrics.f1_score(y_test, y_predRF))

print()
print('confusion matrix')
print(cm)
print()

# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = RF, X = X_train, y = y_train, cv = 10)
print('accuracies mean',accuracies.mean())
print('accuracies with 2 SD', accuracies.std() * 2)
print()


# Applying Grid Search to find the best model and the best parameters
from sklearn.model_selection import GridSearchCV

parameters = [{'criterion': ["entropy"], 'max_depth': [10,20,40,60,80,100,120], 'min_samples_split': [20,40,60,80], 'n_estimators':[10,20,30,40], 'bootstrap':[False]}]
grid_search = GridSearchCV(estimator = RF,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1, verbose=1)
grid_search = grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_


from sklearn.metrics import classification_report
report =classification_report(y_test, y_predRF)

print()
print('classification report')
print(report)
print()

print('best accuracy', best_accuracy)
print('best parameters')
print(best_parameters)
print()

print('feature importances')
print(features_and_target_to_use_2_labels.columns)
print(RF.feature_importances_)
print()


In [ ]:
# taking features
X = features_and_target_to_use_2_labels.iloc[:, :-1].values
y = features_and_target_to_use_2_labels.iloc[:, 7].values

# Encoding categorical data
# Encoding the Independent Variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()
X[:, 0] = labelencoder_X.fit_transform(X[:, 0])
onehotencoder = OneHotEncoder(categorical_features = [0])
X = onehotencoder.fit_transform(X).toarray()

from imblearn.over_sampling import SMOTE
import numpy as np
# observing the counts of positive and negative before SMOTE
unique_elements, counts_elements = np.unique(y, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))
# applying SMOTE
smt = SMOTE()
X, y = smt.fit_sample(X, y)
# checking the rows
# observing the counts of positive and negative after SMOTE
unique_elements, counts_elements = np.unique(y, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))


# splitiing the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


##Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)


##Logistic Regression
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(C = 0.1, multi_class='ovr', penalty='l1', solver= 'liblinear')
LR.fit(X_train, y_train)
y_predLR = LR.predict(X_test)


# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predLR)


import sklearn.metrics
print('Accuracy',sklearn.metrics.accuracy_score(y_test, y_predLR))
#print('Precision',sklearn.metrics.precision_score(y_test, y_predLR))
#print('Recall',sklearn.metrics.recall_score(y_test, y_predLR))
#print('F-1 score',sklearn.metrics.f1_score(y_test, y_predLR))

print()
print('confusion matrix')
print(cm)
print()

# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = LR, X = X_train, y = y_train, cv = 10)
accuracies.mean()

# Applying Grid Search to find the best model and the best parameters
from sklearn.model_selection import GridSearchCV

parameters = [{'penalty': ["l1"], 'C': [0.1,0.2,0.5,0.8,1,1.2,1.4], 'solver':['liblinear','saga'], 'multi_class':['ovr']},
             {'penalty': ["l2"], 'C': [0.1,0.2,0.5,0.8,1,1.2,1.4], 'solver':['newton-cg','lbfgs','sag'], 'multi_class':['ovr', 'multinomial']}
             ]
grid_search = GridSearchCV(estimator = LR,
                          param_grid = parameters,
                          scoring = 'accuracy',
                          cv = 10,
                          n_jobs = -1, verbose=1)
grid_search = grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_


# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = LR, X = X_train, y = y_train, cv = 10)
print('accuracies mean',accuracies.mean())
print('accuracies with 2 SD', accuracies.std() * 2)
print()

from sklearn.metrics import classification_report
report =classification_report(y_test, y_predLR)

print()
print('classification report')
print(report)
print()

print('best accuracy', best_accuracy)
print('best parameters')
print(best_parameters)
print()


In [ ]:
# taking features
X = features_and_target_to_use_2_labels.iloc[:, :-1].values
y = features_and_target_to_use_2_labels.iloc[:, 7].values

# Encoding categorical data
# Encoding the Independent Variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()
X[:, 0] = labelencoder_X.fit_transform(X[:, 0])
onehotencoder = OneHotEncoder(categorical_features = [0])
X = onehotencoder.fit_transform(X).toarray()

from imblearn.over_sampling import SMOTE
import numpy as np
# observing the counts of positive and negative before SMOTE
unique_elements, counts_elements = np.unique(y, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))
# applying SMOTE
smt = SMOTE()
X, y = smt.fit_sample(X, y)
# checking the rows
# observing the counts of positive and negative after SMOTE
unique_elements, counts_elements = np.unique(y, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))


# splitiing the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


##Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)


# Fitting Naive Bayes to the Training set
from sklearn.naive_bayes import GaussianNB
NB = GaussianNB()
NB.fit(X_train, y_train)

# Predicting the Test set results
y_predNB = NB.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predNB)


import sklearn.metrics
print('Accuracy',sklearn.metrics.accuracy_score(y_test, y_predNB))
#print('Precision',sklearn.metrics.precision_score(y_test, y_predNB))
#print('Recall',sklearn.metrics.recall_score(y_test, y_predNB))
#print('F-1 score',sklearn.metrics.f1_score(y_test, y_predNB))

print()
print('confusion matrix')
print(cm)
print()

# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = NB, X = X_train, y = y_train, cv = 10)
print('accuracies mean',accuracies.mean())
print('accuracies with 2 SD', accuracies.std() * 2)
print()

from sklearn.metrics import classification_report
report =classification_report(y_test, y_predNB)

print()
print('classification report')
print(report)
print()


In [ ]:
# taking features
X = features_and_target_to_use_2_labels.iloc[:, :-1].values
y = features_and_target_to_use_2_labels.iloc[:, 7].values

# Encoding categorical data
# Encoding the Independent Variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()
X[:, 0] = labelencoder_X.fit_transform(X[:, 0])
onehotencoder = OneHotEncoder(categorical_features = [0])
X = onehotencoder.fit_transform(X).toarray()

from imblearn.over_sampling import SMOTE
import numpy as np
# observing the counts of positive and negative before SMOTE
unique_elements, counts_elements = np.unique(y, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))
# applying SMOTE
smt = SMOTE()
X, y = smt.fit_sample(X, y)
# checking the rows
# observing the counts of positive and negative after SMOTE
unique_elements, counts_elements = np.unique(y, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

# splitiing the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


##Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)


# Fitting K-NN to the Training set
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors= 160, algorithm= 'auto')
KNN.fit(X_train, y_train)

# Predicting the Test set results
y_predKNN = KNN.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predKNN)


import sklearn.metrics
print('Accuracy',sklearn.metrics.accuracy_score(y_test, y_predKNN))
#print('Precision',sklearn.metrics.precision_score(y_test, y_predKNN))
#print('Recall',sklearn.metrics.recall_score(y_test, y_predKNN))
#print('F-1 score',sklearn.metrics.f1_score(y_test, y_predKNN))

print()
print('confusion matrix')
print(cm)
print()

# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = KNN, X = X_train, y = y_train, cv = 10)
print('accuracies mean',accuracies.mean())
print('accuracies with 2 SD', accuracies.std() * 2)
print()

# Applying Grid Search to find the best model and the best parameters
from sklearn.model_selection import GridSearchCV

parameters = [{'n_neighbors': [5,10,20,25,30,40,50,100,120,140,160,180,200,220,240], 'algorithm': ['ball_tree','kd_tree']}]
grid_search = GridSearchCV(estimator = KNN,
                          param_grid = parameters,
                          scoring = 'accuracy',
                          cv = 10,
                          n_jobs = -1, verbose=1)
grid_search = grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_


from sklearn.metrics import classification_report
report =classification_report(y_test, y_predKNN)

print()
print('classification report')
print(report)
print()

print('best accuracy', best_accuracy)
print('best parameters')
print(best_parameters)
print()


In [ ]:
"""PLOTTING"""
'''
from sklearn import metrics
import matplotlib.pyplot as plt

plt.figure(0).clf()

plt.figure(figsize=(7,7))

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")

x = [0.0, 1.0]
plt.plot(x, x, linestyle='dashed', color='red', linewidth=2, label='random')

#RF
fpr, tpr, thresh = metrics.roc_curve(y_test, y_predRF)
auc = metrics.roc_auc_score(y_test, y_predRF)
plt.plot(fpr,tpr,label="Random Forest, auc="+str(auc)[:5])

#DT
fpr, tpr, thresh = metrics.roc_curve(y_test, y_predDT)
auc = metrics.roc_auc_score(y_test, y_predDT)
plt.plot(fpr,tpr,label="Decision Tree, auc="+str(auc)[:5])

#KNN
fpr, tpr, thresh = metrics.roc_curve(y_test, y_predKNN)
auc = metrics.roc_auc_score(y_test, y_predKNN)
plt.plot(fpr,tpr,label="KNN, auc="+str(auc)[:5])

#LR
fpr, tpr, thresh = metrics.roc_curve(y_test, y_predLR)
auc = metrics.roc_auc_score(y_test, y_predLR)
plt.plot(fpr,tpr,label="Logistic Regression, auc="+str(auc)[:5])

#NB
fpr, tpr, thresh = metrics.roc_curve(y_test, y_predNB)
auc = metrics.roc_auc_score(y_test, y_predNB)
plt.plot(fpr,tpr,label="Naive Bayes, auc="+str(auc)[:5])

plt.legend(loc='lower right')

plt.show()
'''